This notebook is part of the Lecture on Weather Radar at Institute for Geosciences, Meteorology Section, University of Bonn.

https://git.meteo.uni-bonn.de/projects/radarmeteorology

Copyright (c) 2018-2022, Institute for Geosciences, Meteorology Section.

# Attenuation Correction - ZPHI-Method

Testud, J., Le Bouar, E., Obligis, E., & Ali-Mehenni, M. (2000). The Rain Profiling Algorithm Applied to Polarimetric Weather Radar, Journal of Atmospheric and Oceanic Technology, 17(3), 332-356. Retrieved Nov 24, 2021, from https://journals.ametsoc.org/view/journals/atot/17/3/1520-0426_2000_017_0332_trpaat_2_0_co_2.xml

Diederich, M., Ryzhkov, A., Simmer, C., Zhang, P., & Trömel, S. (2015). Use of Specific Attenuation for Rainfall Measurement at X-Band Radar Wavelengths.: Part I: Radar Calibration and Partial Beam Blockage Estimation. Journal of Hydrometeorology, 16(2), 487–502. http://www.jstor.org/stable/24914953



In [ ]:
import wradlib as wrl
import matplotlib.pyplot as pl
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')
try:
    get_ipython().magic("matplotlib inline")
except:
    pl.ion()
import numpy as np
import os
import sys
import glob
import datetime as dt

In [ ]:
import xarray as xr
import itertools
#import hvplot
#import hvplot.xarray

In [ ]:
#sys.path.insert(0, '../')
#from radarmet import *

### Plot Some Data

##### Setup event

- stratiform event
  - start_time = dt.datetime(2016, 3, 31, 18, 10)

- convective event
  - start_time = dt.datetime(2011, 6, 22, 11, 20)

In [ ]:
fglob = "/automount/hubhome/k.muehlbauer/erad2022/data/meteoswiss/2019*.vol"
fglob = "/home/kai/daten/daten/radar_all_over_the_world/iris_jorma/SUR210912_refl/SUR210912080007.RAW3EJZ"

In [ ]:
flist = sorted(glob.glob(fglob))
sep = flist[0].split('/')[-1][:11]
sep

In [ ]:
nflist = []
iterator = itertools.groupby(sorted(flist), lambda f: f.split('/')[-1][:11])
for element, group in iterator:
    nflist.append(list(group))

nflist[-1]

In [ ]:
%%time
vol = wrl.io.open_iris_dataset(fglob, reindex_angle=1.0)

In [ ]:
display(vol)

In [ ]:
swp = vol[0].copy()
swp = swp.pipe(wrl.georef.georeference_dataset)

In [ ]:
display(swp)

#### Create Plot

In [ ]:
fig = pl.figure(figsize=(13,5))

ax1 = fig.add_subplot(121)
im1 = swp.RHOHV.plot(x="x", y="y", ax=ax1, cmap="turbo")
t = pl.title(r'Uncorrected $\phi_{DP}$')
t.set_y(1.1)

ax2 = fig.add_subplot(122)
im2 = swp.DBTH.plot(x="x", y="y", ax=ax2, cmap="turbo", vmin=-10, vmax=50)
t = pl.title(r'Uncorrected $Z_{H}$')
t.set_y(1.1)
fig.suptitle(swp.time.values, fontsize=14)
fig.subplots_adjust(wspace=0.25)

Preprocessed $PHI_{DP}$ is needed (See [Lesson3](../Lesson_03/Phase_Processing.ipynb)).

### Pre-Processing

The phase data is filtered using a median filter and smoothed with a gaussian kernel. Finally offset correction is applied.

#### Copy Dataset

In [ ]:
# copy moments
swp_msk = swp.copy()
phiraw = swp.PHIDP.copy()

dr_m = swp.range.diff('range').median()
dr_km = dr_m / 1000.
scantime = swp.time.values.astype('<M8[s]')
scantime

In [ ]:
dr_m, dr_km

#### Threshold, Filter and Smooth, Offset Retrieval

Do not remove too much data for phase retrieval, it's needed to correctly retrieve phase values in far away regions and/or where RHOHV is low due to clutter and other effects.

In [ ]:
# mask uh and rho
swp_msk = swp.where((swp.DBZH >= 0.))
swp_msk = swp_msk.where(swp_msk.RHOHV > 0.70)
swp_msk = swp_msk.where(swp_msk.range > dr_m * 5)

In [ ]:
swp_msk.DBZH.plot(x="x", y="y", cmap="turbo")

In [ ]:
phi_masked = swp_msk.PHIDP.copy()

In [ ]:
# median filtering 2d
import scipy
def filter_data(data, medwin):
    data.values = scipy.signal.medfilt2d(data.values, [1, medwin])
    return data

medwin = 7
phimed = filter_data(phi_masked, medwin=medwin)

In [ ]:
phimed.plot()

In [ ]:
def convolve(data, kernel, mode='same'):
    mask = np.isnan(data)
    out = np.convolve(np.where(mask, 0, data), kernel, mode=mode) / np.convolve(~mask, kernel, mode=mode)
    return out

def gauss_kernel(width, sigma):
    dirac = np.zeros(width)
    dirac[int(width / 2)] = 1
    return scipy.ndimage.gaussian_filter1d(dirac, sigma=sigma)

def smooth_data(data, kernel):
    res = data.copy()
    for i, dat in enumerate(data.values):
        res[i] = convolve(dat, kernel)
    return res


def phase_offset(phioff, rng=3000.):
    """Calculate Phase offset.

    Parameter
    ---------
    phioff : xarray.DataArray
        differential phase array

    Keyword Arguments
    -----------------
    rng : float
        range in m to calculate system phase offset

    Return
    ------
    start_range : xarray.DataArray
        DataArray with start range values
    off : xarray.DataArray
        DataArray with phase offset values
    """
    range_step = np.diff(phioff.range)[0]
    nprec = int(rng / range_step)
    if nprec % 2:
        nprec += 1

    # create binary array
    phib = xr.where(np.isnan(phioff), 0, 1)

    # take nprec range bins and calculate sum
    phib_sum = phib.rolling(range=nprec, center=True).sum(skipna=True)

    # get start range of first N consecutive precip bins
    start_range = phib_sum.idxmax(dim="range") - nprec // 2 * np.diff(phib_sum.range)[0]
    # get range of first non-nan value per ray
    #start_range = (~np.isnan(phioff)).idxmax(dim='range', skipna=True)
    # add range
    stop_range = start_range + rng
    # get phase values in specified range
    off = phioff.where((phioff.range >= start_range) & (phioff.range <= stop_range),
                       drop=True)
    # calculate nan median over range
    off = off.median(dim='range', skipna=True)
    return xr.Dataset(dict(PHIDP_OFFSET=off,
                           start_range=start_range,
                           stop_range=stop_range))

#print(phimed.shape)
#phimed = np.ma.array(phimed, mask=res_mask)

# gaussian convolution 1d - smoothing
# play with sigma and kwidth
kwidth = 7
sigma = 10
gkern = gauss_kernel(kwidth, sigma)
phiclean = smooth_data(phimed, gkern)
#phiclean = np.ma.array(phiclean, mask=res_mask)

# median over all first range bins, broadcasted to all rays
off = phase_offset(phiclean, 3000.)
phi_offset1 = off.PHIDP_OFFSET.load().median(dim='azimuth', skipna=True)
phicorr = phiclean - phi_offset1

In [ ]:
phi_offset1

In [ ]:
phiclean.plot()

In [ ]:
phicorr.plot()

In [ ]:
#off = phase_offset(swp.PHIDP, 3000.)
#phi_offset1 = off.PHIDP_OFFSET.load().median(dim='azimuth', skipna=True)
#phicorr = phiclean - phi_offset1

In [ ]:
swp.PHIDP.plot()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,10))
ax1 = plt.subplot(111, projection='polar')
# set the lable go clockwise and start from the top
ax1.set_theta_zero_location("N")
# clockwise
ax1.set_theta_direction(-1)
theta = np.linspace(0, 2*np.pi, num=360, endpoint=False)
ax1.plot(theta, off.PHIDP_OFFSET, color='b', linewidth=3)
ax1.plot(theta, np.ones_like(theta) * phi_offset1.values, color='r', lw=2)
ti = off.time.values.astype('M8[s]')
om = phi_offset1.values
tx = ax1.set_title(f'{ti}\n' + r'$\phi_{DP}-Offset$ ' + f'{om:.1f} (deg)')
tx.set_y(1.1)
xticks = ax1.set_xticks(np.pi/180. * np.linspace(0,  360, 36, endpoint=False))
#ax1.set_ylim([-100, -80])

#### Check radar domain

In [ ]:
def get_discrete_cmap(ticks, colors, bad="white", over=None, under=None):
    """Create discrete colormap.

    Parameters
    ----------
    ticks : int | sequence
        number of ticks or sequence of ticks
    colors : colormap | sequence
        colormap or sequence of colors
    bad : color
    over : color
    under : color

    Returns
    -------
    matplotlib.colors.ListedColormap
    """
    ticks = ticks if isinstance(ticks, int) else len(ticks)
    if isinstance(colors, (str, mpl.colors.Colormap)):
        cmap = mpl.cm.get_cmap(colors)
        colors = cmap(np.linspace(0, 1, ticks + 1))
    cmap = mpl.colors.ListedColormap(colors[1:-1])
    if over is None:
        over = colors[-1]
    if under is None:
        under = colors[0]
    cmap.set_under(under)
    cmap.set_over(over)
    cmap.set_bad(color=bad)
    return cmap


def get_discrete_norm(ticks):
    """Return discrete boundary norm.

    Parameters
    ----------
    ticks : sequence
        sequence of ticks

    Returns
    -------
    matplotlib.colors.BoundaryNorm
    """
    return mpl.colors.BoundaryNorm(ticks, len(ticks) - 1)

In [ ]:
swp.PHIDP[:, 0:10].plot()

In [ ]:
fig = pl.figure(figsize=(10,8))
t = fig.suptitle(scantime, fontsize=14)
t.set_y(0.99)
ticks1 = np.arange(0, 160, 10)
norm1 = get_discrete_norm(ticks1)
cmap = mpl.cm.get_cmap('turbo')
im1 = (swp.PHIDP - phi_offset1).wradlib.plot(ax=221, fig=fig,
                                           cmap=cmap, norm=norm1)
t = pl.title(r'Uncorrected $\phi_{DP}$')
t.set_y(1.1)

im2 = (swp_msk.PHIDP - phi_offset1).wradlib.plot(ax=222, fig=fig, 
                                        cmap=cmap, norm=norm1)
t = pl.title(r'Thresholded $\phi_{DP}$')
t.set_y(1.1)

im3 = phicorr.wradlib.plot(ax=223, fig=fig, 
                           cmap=cmap, norm=norm1)
t = pl.title(r'Corrected $\phi_{DP}$')
t.set_y(1.1)

# add colorbar
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.1, 0.025, 0.8])
cbar = fig.colorbar(im3, cax=cbar_ax, norm=norm1, label=r'$\phi_{DP}$ (deg)')
cbar.locator = mpl.ticker.FixedLocator(ticks1)
cbar.update_ticks()
# make some space between subplots
fig.subplots_adjust(wspace=0.3, hspace=0.3)
pl.show()

### ZPHI-Method

see Testud et.al. (chapter 4. p. 339ff.), Diederich et.al. (chapter 3. p. 492 ff).

There is a equational difference in the two papers, which can be solved like this:

$\begin{equation}
f\Delta\phi_{DP} = 10^{0.1 \cdot b \cdot \alpha \cdot \Delta\phi_{DP}} - 1
\tag{1}
\end{equation}$

$\begin{equation}
C(b, PIA) = \exp[{0.23 \cdot b \cdot (PIA)}] - 1
\tag{2}
\end{equation}$

with 

$\begin{equation}
PIA = \alpha \cdot \Delta\phi_{DP}
\tag{3}
\end{equation}$

$\begin{equation}
C(b, PIA) = \exp[{0.23 \cdot b \cdot \alpha \cdot \Delta\phi_{DP}}] - 1
\tag{4}
\end{equation}$

Both expressions are used equivalently:

$\begin{equation}
10^{0.1 \cdot b \cdot \alpha \cdot \Delta\phi_{DP}} - 1 = \exp[{0.23 \cdot b \cdot \alpha \cdot \Delta\phi_{DP}}] - 1
\tag{5}
\end{equation}$


Using logarithmic identities:

$\begin{equation}
\ln {u^r} = r \cdot \ln {u}
\tag{6a}
\end{equation}$

$\begin{equation}
\exp {\ln x} = x
\tag{6b}
\end{equation}$

the left hand side can be further expressed as:

$\begin{equation}
\exp [\ln {10^{0.1 \cdot b \cdot \alpha \cdot \Delta\phi_{DP}}}] - 1 = \exp[{0.23 \cdot b \cdot \alpha \cdot \Delta\phi_{DP}}] - 1
\tag{7a}
\end{equation}$


$\begin{equation}
\exp[0.1 \cdot b \cdot \alpha \cdot \Delta\phi_{DP} \cdot \ln {10}] - 1 = \exp[{0.23 \cdot b \cdot \alpha \cdot \Delta\phi_{DP}}] - 1
\tag{7b}
\end{equation}$

leading to equality

$\begin{equation}
\exp[0.23 \cdot b \cdot \alpha \cdot \Delta\phi_{DP}] - 1 = \exp[{0.23 \cdot b \cdot \alpha \cdot \Delta\phi_{DP}}] - 1
\tag{7c}
\end{equation}$

We are using the following quations derived from the above papers:

$\begin{equation}
\Delta \phi_{DP} = \phi_{DP}(r2) - \phi_{DP} (r1)
\tag{1}
\end{equation}$

$\begin{equation}
f\Delta\phi_{DP} = 10^{0.1 \cdot b \cdot \alpha \cdot \Delta\phi_{DP}} - 1
\tag{2}
\end{equation}$

$\begin{equation}
A_{H}(r) = \frac{\left [Z_a(r) \right ]^b \cdot f(\Delta \phi_{DP})}{0.46b \int_{r1}^{r2} \left [Z_a(s) \right ]^b ds + f(\Delta \phi_{DP}) \cdot 0.46b \int_{r}^{r2} \left [Z_a(s) \right ]^b ds}
\tag{3}
\end{equation}$

$\begin{equation}
\phi_{DP}^{cal}(r_i, \alpha) = 2 \cdot \int_{r1}^{r2} \frac{A_H(s; \alpha)}{\alpha}ds
\tag{4}
\end{equation}$

$\begin{equation}
\Delta = \sum_{i=1}^{N} \left|{\phi_{DP}^{cal}(r_i, \alpha, b)- \phi_{DP}^{m}(r_i)}\right|
\tag{5}
\end{equation}$

$\begin{equation}
A_{H} = \beta Z_{H}^{b}
\tag{6}
\end{equation}$

$\begin{equation}
A_{H} = \alpha K_{DP}
\tag{7}
\end{equation}$

#### Retrieving $\Delta \phi_{DP}$

We will use the simple method of finding the first and the last non NAN values per ray from $\phi_{DP}^{corr}$.

This is the most simple and probably not very robust method.

In [ ]:
# find first occurrence of non NAN phi per ray
first = np.nanargmax(~np.isnan(phiclean), axis=1)
# find last occurrence of non NAN phi per ray
last = phiclean.shape[1] - np.nanargmax(np.flip(~np.isnan(phiclean), axis=-1), axis=1) - 1

In [ ]:
# delta phi
dphi_old = np.array([(phiclean[i, last[i]] - phiclean[i, first[i]]) 
                 for i in range(phiclean.shape[0])])

### Use range of values to extract first/last PHIDP values

In [ ]:
def phase_zphi(phi, rng=1000.):
    range_step = np.diff(phi.range)[0]
    nprec = int(rng / range_step)
    if nprec % 2:
        nprec += 1

    # create binary array
    phib = xr.where(np.isnan(phi), 0, 1)
    
    # take nprec range bins and calculate sum
    phib_sum = phib.rolling(range=nprec, center=True).sum(skipna=True)
    
    offset = nprec // 2 * np.diff(phib_sum.range)[0]
    offset_idx = nprec // 2
    start_range = phib_sum.idxmax(dim="range") - offset
    start_range_idx = phib_sum.argmax(dim="range") - offset_idx
    stop_range = phib_sum[:, ::-1].idxmax(dim="range") - offset
    stop_range_idx = len(phib_sum.range) - (phib_sum[:, ::-1].argmax(dim="range") - offset_idx) - 2
    # get phase values in specified range
    first = phi.where((phi.range >= start_range) & (phi.range <= start_range + rng),
                       drop=True).min(dim='range', skipna=True)
    last = phi.where((phi.range >= stop_range - rng) & (phi.range <= stop_range),
                       drop=True).max(dim='range', skipna=True)
    
    
    return xr.Dataset(dict(phib=phib_sum,
                           offset=offset,
                           offset_idx=offset_idx,
                           start_range=start_range,
                           stop_range=stop_range,
                           first=first,
                           first_idx=start_range_idx,
                           last=last,
                           last_idx=stop_range_idx,
                          ))


In [ ]:
fig = pl.figure(figsize=(10,8))
cphase = phase_zphi(phiclean, rng=1000.)
cphase.first.plot(label="first")
cphase.last.plot(label="last")
dphi = cphase.last - cphase.first
dphi = dphi.where(dphi>=0).fillna(0)
dphi.plot(ls='-', marker='.', label='delta')
pl.gca().grid()
pl.legend()

#### $\Delta \phi_{DP}$ - Polar Plots

This visualizes `first` and `last` indizes including $\Delta \phi_{DP}$.

In [ ]:
fig = pl.figure(figsize=(20, 9))
ax1 = pl.subplot(131, projection='polar')
ax2 = pl.subplot(132, projection='polar')
ax3 = pl.subplot(133, projection='polar')
# set the lable go clockwise and start from the top
ax1.set_theta_zero_location("N")
ax2.set_theta_zero_location("N")
ax3.set_theta_zero_location("N")
# clockwise
ax1.set_theta_direction(-1)
ax2.set_theta_direction(-1)
ax3.set_theta_direction(-1)
theta = np.linspace(0, 2*np.pi, num=360, endpoint=False)
ax1.plot(theta, cphase.first_idx, color='b', linewidth=3)
ax1.plot(theta, first, color='k', linewidth=1)
_ = ax1.set_title("Delta PHIDP - First Index")
ax2.plot(theta, cphase.last_idx, color='r', linewidth=3)
ax2.plot(theta, last, color='k', linewidth=1)
_ =  ax2.set_title("Delta PHIDP - Last Index")
ax3.plot(theta, dphi, color='g', linewidth=3)
ax3.plot(theta, dphi_old, color='k', linewidth=1)
_ =  ax3.set_title("Delta PHIDP")

#### Calculating $f\Delta\phi_{DP}$

$$f\Delta\phi_{DP} = 10^{0.1 \cdot b \cdot \alpha \cdot \Delta\phi_{DP}} - 1$$

In [ ]:
print(dphi.shape)
dphi = dphi.values

In [ ]:
alphax = 0.28
betax = 0.05
bx = 0.78
# need to expand alphax to dphi-shape
fdphi = 10 ** (0.1 * bx * alphax * dphi) - 1
print(fdphi.shape)

#### Calculating Reflectivity Integrals/Sums

$$za(r) = \left[Z_a(r) \right ]^b$$

$$iza(r,r2) = 0.46 \cdot b \cdot \int_{r}^{r2} \left [Z_a(s) \right ]^b ds$$

We do not restrict (mask) the reflectivities for now, but switch between `DBTH` and `DBZH` to see the difference.

In [ ]:
zhraw = swp.DBZH.where((swp.range > cphase.start_range) & (swp.range < cphase.stop_range))
zhraw.plot(cmap="viridis", vmin=-10, vmax=100)

In [ ]:
#zhraw = zhraw.rolling(dict(range=25), center=True, min_periods=2).mean(skipna=True)
#zhraw.plot(cmap="viridis", vmin=-10, vmax=100)

In [ ]:
# calculate linear reflectivity and ^b
zax = zhraw.pipe(wrl.trafo.idecibel).fillna(0)
zh_cal = zax.pipe(wrl.trafo.decibel)
zh_cal.plot()

In [ ]:
za = zax ** bx
za.plot(vmax=1000)

In [ ]:
# set masked to zero for integration
za_zero = za.fillna(0)
za_zero.wradlib.plot()

Calculate cumulative integral, and subtract from maximum. That way we have the cumulative sum for every bin until the end of the ray.

In [ ]:
from scipy.integrate import cumtrapz
iza_x = 0.46 * bx * cumtrapz(za_zero.values, axis=1, initial=0, dx=dr_km.values)
iza = np.max(iza_x, axis=1)[:, None] - iza_x
print(iza.shape)
pl.figure()
pl.imshow(iza)
iza

#### Calculating Attenuation $A_{H}$ for whole domain

$$A_{H}(r) = \frac{\left [Z_a(r) \right ]^b \cdot f(\Delta \phi_{DP})}{0.46b \int_{r1}^{r2} \left [Z_a(s) \right ]^b ds + f(\Delta \phi_{DP}) \cdot 0.46b \int_{r}^{r2} \left [Z_a(s) \right ]^b ds}$$

We can reduce the number of operations by rearranging the equation like this:

$$A_{H}(r) = \frac{\left [Z_a(r) \right ]^b}{\frac{0.46b \int_{r1}^{r2} \left [Z_a(s) \right ]^b ds}{f(\Delta \phi_{DP})} + 0.46b \int_{r}^{r2} \left [Z_a(s) \right ]^b ds}$$

In [ ]:
iza.shape, fdphi.shape, fdphi[:, np.newaxis].shape

In [ ]:
iza_fdphi = iza / fdphi[..., np.newaxis]
iza_first = np.array([iza_fdphi[ray, cphase.first_idx[ray]] for ray in range(za.shape[0])])

In [ ]:
print(za_zero.shape, iza_first.shape, iza.shape)

The following calculation is **very** slow. Only run this, when you know what you're doing.

In [ ]:
%%time
ah_slow = np.zeros_like(za) * np.nan
for ray in range(za.shape[0]):
    for rbin in range(first[ray], last[ray]):
        ah_slow[ray, rbin] = za[ray, rbin] / (iza_first[ray] + 
                                              iza[ray, rbin])

We can still improve significantly by removing the inner for-loop due to inherent broadcasting:

In [ ]:
%%time
ah = np.zeros_like(za)
for ray in range(za.shape[0]):
    ah[ray] = (za[ray] / 
                      (iza_first[ray] + iza[ray]))

Finally, we can remove the outer loop, too!

In [ ]:
print(iza_first.shape)

In [ ]:
%%time
ah_fast = za / (iza_first[:, None] + iza)

check equality: while the two faster methods give equal output, some elements of the slow implementation differ.

In [ ]:
np.testing.assert_array_equal(ah_fast, ah)
try:
    np.testing.assert_array_equal(np.nan_to_num(ah_slow), np.nan_to_num(ah))
except AssertionError as e:
    print(e)

In [ ]:
fig = pl.figure(figsize=(15, 6))
ticks_ah = np.arange(0, 5, 0.2)
cmap = get_discrete_cmap(ticks_ah, mpl.cm.get_cmap("turbo"))
norm1 = get_discrete_norm(ticks_ah)
ax1, im1 = wrl.vis.plot_ppi(ah_fast, ax=131, fig=fig, norm=norm1, cmap=cmap)
pl.colorbar(im1, norm=norm1, cmap=cmap)
ax2, im2 = wrl.vis.plot_ppi(ah_slow, ax=132, fig=fig, norm=norm1, cmap=cmap)
pl.colorbar(im2, norm=norm1, cmap=cmap)
ax3, im3 = wrl.vis.plot_ppi(ah, ax=133, fig=fig, norm=norm1, cmap=cmap)
pl.colorbar(im3, norm=norm1, cmap=cmap)

#### Calculate $\phi_{DP}^{cal}(r, \alpha)$ for whole domain

$$\phi_{DP}^{cal}(r_i, \alpha) = 2 \cdot \int_{r1}^{r2} \frac{A_H(s; \alpha)}{\alpha}ds$$

filling nan with zero

In [ ]:
ah_fast = np.ma.masked_invalid(ah_fast).filled(0)
ah = np.ma.masked_invalid(ah).filled(0)

In [ ]:
phical = 2 * cumtrapz(ah/alphax, axis=1, dx=dr_km.values, initial=0)
phical_fast = 2 * cumtrapz(ah_fast/alphax, axis=1, dx=dr_km.values, initial=0)

In [ ]:
def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for _, sp in ax.spines.items():
        sp.set_visible(False)


def set_spine_direction(ax, direction):
    if direction in ["right", "left"]:
        ax.yaxis.set_ticks_position(direction)
        ax.yaxis.set_label_position(direction)
    elif direction in ["top", "bottom"]:
        ax.xaxis.set_ticks_position(direction)
        ax.xaxis.set_label_position(direction)
    else:
        raise ValueError("Unknown Direction: %s" % (direction,))

    ax.spines[direction].set_visible(True)

def create_lineplot(fig, subplot=111, xlabel=None, ylabel=None):

    if xlabel is None:
        xlabel = 'Range Bins'

    if ylabel is None:
        ylabel = ''

    host = fig.add_subplot(subplot)

    ax = host
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    return ax


def add_axis(host, ylabel=None, pos=1.0):
    if ylabel is None:
        ylabel=''
    ax = host.twinx()
    host.spines["right"].set_visible(False)
    ax.spines["right"].set_position(("axes", pos))
    make_patch_spines_invisible(ax)
    set_spine_direction(ax, "right")
    ax.set_ylabel(ylabel)

    return ax


In [ ]:
# 50, 70, 80, 90, 180, 270, 300
az = 90
azi = np.arange(az, az + 1)
fig = pl.figure(figsize=(10,6))
ax0 = create_lineplot(fig, subplot=111, ylabel=r"$PHI_{DP}$ (deg)")
ax0.set_ylim(-10, 180)
#ax0.set_xlim(175, 600)
t = ax0.set_title(f"{swp.time.values.astype('<M8[s]')}")
t.set_y(1.05)
line = ax0.plot(phiraw[azi,:].T - phi_offset1, 'b.', label='$PHI_{DP}^{raw}$')
line = ax0.plot(phiclean[azi,:].T - phi_offset1, 'r', label='$PHI_{DP}^{corr}$')
line = ax0.plot(phical_fast[azi,:].T, 'k', label='$PHI_{DP}^{corr+offset}$')
#ax0.plot(cphase.)
ax0.grid()
pl.legend(ncol=4, loc=8)

# add twin axis of host axis with label and position
ax1 = add_axis(ax0, ylabel=r'$UZ_H$ (dBZ)', pos=1.0)
#line = ax1.plot(swp.DBTH[azi,:].T, 'g.-', zorder=0)
line = ax1.plot(zh_cal[azi,:].T, 'g.-', zorder=0)
line = ax1.plot(swp_msk.DBTH[azi,:].T, 'k.', zorder=0)


# add another twin axis of host axis with label and position
ax2 = add_axis(ax0, ylabel=r'$RHO_{HV}$', pos=1.1)
line = ax2.plot(swp.RHOHV[azi,:].T, 'r.-')
line = ax2.plot(swp_msk.RHOHV[azi,:].T, 'c.')
ax2.set_ylim(0.7, 1.0)
pl.tight_layout()
pl.show()

In [ ]:
phicalx = xr.full_like(phiraw, np.nan)
phicalx.data = phical_fast
phicalx = phicalx.rename("PHICAL")
phicalx
phiout = xr.merge([phiraw-cphase.first.values[:, None], phicalx])
phiout

In [ ]:
phiout.PHICAL.plot()

In [ ]:
#import holoviews as hv
#hv.output(widget_location='bottom')
#phi_plot = phiout.hvplot.line(groupby="azimuth", ylim=(-10, 300))
#phi_plot

In [ ]:
fig = pl.figure(figsize=(12, 4))
cmap = mpl.cm.get_cmap('turbo')
ticks_ah = np.arange(0, 180, 10)
norm1 = get_discrete_norm(ticks_ah)
ax1, im1 = wrl.vis.plot_ppi(phical_fast, ax=121, fig=fig, norm=norm1, cmap=cmap)
pl.colorbar(im1, norm=norm1, cmap=cmap)
ax3, im3 = wrl.vis.plot_ppi(phical, ax=122, fig=fig, norm=norm1, cmap=cmap)
pl.colorbar(im3, norm=norm1, cmap=cmap)

Check the plot for different azimuth angles and start over from [the beginning](#Calculating-$f\Delta\phi_{DP}$) to iteratively find an adequate value for $\alpha$.

An automated algorithm would try to minimize the [$\Delta$-Sum](#Calculation-of-$\Delta$-Sum) (below).

#### Plotting different $\phi_{DP}$ for Radar Domain

In [ ]:
phi_offset1

In [ ]:
fig = pl.figure(figsize=(10,8))
fig.suptitle(scantime, fontsize=14)

ticks1 = np.arange(0, 210, 10)
cmap = mpl.cm.get_cmap('turbo')

norm1 = get_discrete_norm(ticks1)
im1 = (swp.PHIDP - phi_offset1).wradlib.plot(ax=221, fig=fig, 
                         norm=norm1, cmap=cmap)
t = pl.title(r'Uncorrected $\phi_{DP}$')
t.set_y(1.1)

im2 = (swp_msk.PHIDP - phi_offset1).wradlib.plot(ax=222, fig=fig, 
                            cmap=cmap, norm=norm1)
t = pl.title(r'Thresholded $\phi_{DP}$')
t.set_y(1.1)

fig.subplots_adjust(wspace=0.35, hspace=0.3)

ticks2 = np.arange(-5, 11, 1)
im3 = phicorr.wradlib.plot(ax=223, fig=fig, 
                          norm=norm1, cmap=cmap)
t = pl.title(r'Corrected $\phi_{DP}$')
t.set_y(1.1)

ax4, im4 = wrl.vis.plot_ppi(phical_fast.where(swp_msk.PHIDP), ax=224, fig=fig,                        
                            cmap=cmap, norm=norm1)
t = pl.title(r'Calculated $\phi_{DP}$')
t.set_y(1.1)

# add colorbar
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.1, 0.025, 0.8])
cbar = fig.colorbar(im1, cax=cbar_ax, norm=norm1, label=r'$\phi_{DP}$ (deg)')
cbar.locator = mpl.ticker.FixedLocator(ticks1)
cbar.update_ticks()
# make some space between subplots
fig.subplots_adjust(wspace=0.3, hspace=0.3)
pl.show()

#### Calculation of $\Delta$-Sum
$$\Delta = \sum_{i=1}^{N} \left|{\phi_{DP}^{cal}(r_i, \alpha, b)- \phi_{DP}^{m}(r_i)}\right|$$

In [ ]:
delta = np.zeros(phical.shape[0])
diff = np.abs(phical - phicorr)

for ray in range(phical.shape[0]):
    f = first[ray]
    l = last[ray]
    if l>f:
        delta[ray] = np.sum(diff[ray, f:l])

In [ ]:
fig = pl.figure(figsize=(10,8))
ax1 = pl.subplot(111, projection='polar')
# set the lable go clockwise and start from the top
ax1.set_theta_zero_location("N")
# clockwise
ax1.set_theta_direction(-1)
theta = np.linspace(0, 2*np.pi, num=360, endpoint=False)
ax1.plot(theta, delta, color='b', linewidth=1)
t = ax1.set_title(r"$\Delta$-Sum")
t.set_y(1.1)
#ax1.set_rmax(20000)
xticks = ax1.set_xticks(np.pi/180. * np.linspace(0,  360, 36, endpoint=False))

#### Apply attenuation correction 

Finally, after retrieval of the fitting $\alpha$ we correct $Z_H$ and $Z_{DR}$ for attenuation.

In [ ]:
print(alphax)

In [ ]:
zhraw = swp.DBTH.copy()
zdrraw = swp.ZDR.copy()

In [ ]:
with xr.set_options(keep_attrs=True):
    zhcorr = zhraw + alphax * (phical)
    zdiff = zhcorr - zhraw
    zdrcorr = zdrraw + betax * (phical)
    zdrdiff = zdrcorr - zdrraw

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = pl.subplots(nrows=2, ncols=2, 
                                            figsize=(15, 12), 
                                            sharex=True, sharey=True, 
                                            squeeze=True,
                                            constrained_layout=True)

t = fig.suptitle(scantime, fontsize=14)
t.set_y(1.05)


ticks1 = visdict14["ZH"]["ticks"]
plot_moment(zhraw, ticks1, fig=fig, ax=ax1)
plot_moment(zhcorr, ticks1, fig=fig, ax=ax2)
ax2.set_title(r'Corrected $Z_{H}$', fontsize=16)


ticks2 = visdict14["ZDR"]["ticks"]
plot_moment(zdrraw, ticks2, fig=fig, ax=ax3)
plot_moment(zdrcorr, ticks2, fig=fig, ax=ax4)
ax4.set_title(r'Corrected $Z_{DR}$', fontsize=16)

#### $K_{DP}$ from $A_H$ vs. $K_{DP}$ from $\phi_{DP}$

- $K_{DP} = \frac{A_H}{\alpha}$ 
- $K_{DP} = \frac{1}{2}\frac{\mathrm{d}\phi_{DP}}{\mathrm{d}r}$

What are the benefits of $K_{DP}(A_H)$? 

- no noise artefacts
- no $\delta$
- no negative $K_{DP}$
- no spatial degradation

In [ ]:
%%time
kdp = kdp_from_phidp(phicorr, winlen=31)
kdp.attrs = wrl.io.xarray.moments_mapping["KDP"]
kdp_a = xr.zeros_like(kdp)
kdp_a.attrs = wrl.io.xarray.moments_mapping["KDP"]
kdp_a.data = ah / alphax

In [ ]:
fig, (ax1, ax2) = pl.subplots(nrows=2, ncols=1, 
                              figsize=(10, 18), 
                              constrained_layout=True)

ticks1 = visdict14["KDP"]["ticks"]
plot_moment(kdp, ticks1, ax=ax1, fig=fig)
ax1.set_title(r'$K_{DP}$ from $\phi_{DP}$', fontsize=16)
plot_moment(kdp_a, ticks1, ax=ax2, fig=fig)
ax2.set_title(r'$K_{DP}$ from $A_{H}$', fontsize=16)

In [24]:
import pandas as pd
import xarray as xr
import numpy as np
temp = 15 + 8 * np.random.randn(2, 2, 3)
precip = 10 * np.random.rand(2, 2, 3)
lon = [[-99.83, -99.32], [-99.79, -99.23]]
lat = [[42.25, 42.21], [42.63, 42.59]]

# for real use cases, its good practice to supply array attributes such as
# units, but we won't bother here for the sake of brevity
ds = xr.Dataset(
        {
            "temperature": (["x", "y", "time"], temp),
            "precipitation": (["x", "y", "time"], precip),
        },
        coords={
            "lon": (["x", "y"], lon),
            "lat": (["x", "y"], lat),
            #"time": pd.date_range("2014-09-06", periods=3),
            #"reference_time": pd.Timestamp("2014-09-05"),
        },
    )
ds.temperature.attrs["units"] = "[]"
display(ds)
ds.to_netcdf("test.nc")

<xarray.Dataset>
Dimensions:        (x: 2, y: 2, time: 3)
Coordinates:
    lon            (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat            (x, y) float64 42.25 42.21 42.63 42.59
Dimensions without coordinates: x, y, time
Data variables:
    temperature    (x, y, time) float64 25.69 19.35 16.58 ... 23.17 19.32 26.53
    precipitation  (x, y, time) float64 8.994 1.406 1.801 ... 8.802 4.705 2.498

In [25]:
!h5dump test.nc

HDF5 "test.nc" {
GROUP "/" {
   ATTRIBUTE "_NCProperties" {
      DATATYPE  H5T_STRING {
         STRSIZE 34;
         STRPAD H5T_STR_NULLTERM;
         CSET H5T_CSET_ASCII;
         CTYPE H5T_C_S1;
      }
      DATASPACE  SCALAR
      DATA {
      (0): "version=2,netcdf=4.8.1,hdf5=1.12.1"
      }
   }
   DATASET "lat" {
      DATATYPE  H5T_IEEE_F64LE
      DATASPACE  SIMPLE { ( 2, 2 ) / ( 2, 2 ) }
      DATA {
      (0,0): 42.25, 42.21,
      (1,0): 42.63, 42.59
      }
      ATTRIBUTE "DIMENSION_LIST" {
         DATATYPE  H5T_VLEN { H5T_REFERENCE { H5T_STD_REF_OBJECT }}
         DATASPACE  SIMPLE { ( 2 ) / ( 2 ) }
         DATA {
         (0): (), ()
         }
      }
      ATTRIBUTE "_FillValue" {
         DATATYPE  H5T_IEEE_F64LE
         DATASPACE  SIMPLE { ( 1 ) / ( 1 ) }
         DATA {
         (0): nan
         }
      }
      ATTRIBUTE "_Netcdf4Coordinates" {
         DATATYPE  H5T_STD_I32LE
         DATASPACE  SIMPLE { ( 2 ) / ( 2 ) }
         DATA {
         (0): 0, 1
   

In [26]:


with xr.open_dataset("test.nc", decode_cf=False) as ds:
    display(ds)
    display(ds.temperature)

<xarray.Dataset>
Dimensions:        (x: 2, y: 2, time: 3)
Dimensions without coordinates: x, y, time
Data variables:
    temperature    (x, y, time) float64 ...
    precipitation  (x, y, time) float64 ...
    lon            (x, y) float64 ...
    lat            (x, y) float64 ...

<xarray.DataArray 'temperature' (x: 2, y: 2, time: 3)>
array([[[25.693888, 19.351604, 16.581365],
        [19.449395, 22.493882, 16.796134]],

       [[21.531212, 13.150379, 21.884866],
        [23.170259, 19.315072, 26.53169 ]]])
Dimensions without coordinates: x, y, time
Attributes:
    _FillValue:   nan
    units:        []
    coordinates:  lon lat

In [23]:
print(ds.temperature.attrs)
ds.temperature.attrs["coordinates"] in xr.coding.times.TIME_UNITS

{'_FillValue': nan, 'units': array([], dtype=float64), 'coordinates': 'lon lat'}


False

In [27]:
xr.decode_cf(ds)

<xarray.Dataset>
Dimensions:        (x: 2, y: 2, time: 3)
Coordinates:
    lon            (x, y) float64 ...
    lat            (x, y) float64 ...
Dimensions without coordinates: x, y, time
Data variables:
    temperature    (x, y, time) float64 ...
    precipitation  (x, y, time) float64 ...